In [ ]:

#BEGIN
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', **{ "size":16}) #**{,,'serif':['Palatino']
rc('text', usetex=True)
import numpy.random as rnd
rnd.seed()
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import scipy.linalg as la
from scipy.optimize import brentq as solve
from scipy.integrate import odeint as ode
from scipy.special import gamma
np.set_printoptions(precision = 3, suppress = 1)
from matplotlib import cm
color = cm.get_cmap("Blues")

# Large-N limit: from random matrices to Sachdev-Ye-Kitaev

## Random matrix (second-quantized)

We consider a free fermion model with a large number ($N$) complex fermions $c_1, \dots, c_N$. The Hamiltonian is quadratic: 
$$ H_2 = \sum_{i, j=1}^N J_{ij} c_i^\dagger c_j $$
where $J_{ij}$ are complex Gaussian random variables which have zero mean. The only nonzero covariances are
$$ \overline{J_{ij} J_{ji}} = \frac{J^2}N \,.  $$

By standard fermion path integral techniques, we can map the partition function $Z = \mathrm{Tr}[e^{-\beta H_2}]$ to a path integral:
$$Z = \int \mathcal{D}[\psi_i(\tau), \psi^{*}_i(\tau)] \exp(-S) $$
where the action is 
$$ S = \int \left[ \sum_i \psi_i^* \partial_\tau \psi + \sum_{ij}^N J_{ij} \psi_i^* \psi_j \right] \mathrm{d}\tau$$
where $\psi_i(\tau)$ is a Grassmanian field, with anti-periodic boundary conditions: $\psi_i(\tau) = -\psi_i(\tau+\beta) $.

Assuming that the random couplings do not lead to replica symmetry breaking, we shall consider the annealed ensemble, that is, average over the parititon function directly. 

- By averaging over $J_{ij}$'s, show that
$$ \overline{Z} = \int \mathcal{D}[\psi_i(\tau), \psi^{*}_i(\tau)] \exp(-S) $$
where now the (effective) action becomes
$$ S = \int \sum_i \psi_i^* \partial_\tau \psi \mathrm{d}\tau -  \frac{J^2}{2N} \iint \sum_{ij}^N  \psi_i^*(\tau) \psi_j(\tau) \psi_j^*(\tau') \psi_i(\tau') \mathrm{d}\tau \mathrm{d}\tau' $$
(To avoid introducing too many subscripts, we shall always call the action $S$)

Now we introduce a dynamical Green function $G(\tau,\tau')$ and its Lagrange multiplier $\Sigma(\tau, \tau')$, that is, we insert an identity 
$$1 = \int \mathcal{D}[G, \Sigma] \exp\left( \int   (\sum_i \psi_i(\tau) \psi_i^*(\tau') - N G(\tau,\tau')   ) \Sigma(\tau',\tau) \mathrm{d} \tau \mathrm{d} \tau' \right)$$
into the path integral above. By doing so, we can identify $\sum_i \psi_i(\tau) \psi_i^*(\tau')$ with $N G(\tau,\tau')$ in the action. 

- Check then that the action becomes 
$$ S =  \sum_i \left[ \int  \psi_i^* \partial_\tau \psi_i \mathrm{d}\tau -  \iint \psi_i^*(\tau') \Sigma(\tau', \tau) \psi_i (\tau) \mathrm{d}\tau \mathrm{d}\tau' \right] +  N \iint \left[ - G(\tau,\tau') \Sigma(\tau',\tau) + G(\tau,\tau')G(\tau',\tau) \right] \mathrm{d}\tau \mathrm{d}\tau'  $$

The first term is now $N$ copies of decoupled Grassmanian Gaussian integral. Performing that we get an effective action that is proportional to $N$:
$$  S/N =  -  \ln \det(\partial_\tau - \Sigma) +   \iint \left[ - G(\tau,\tau') \Sigma(\tau',\tau) + \frac{J^2}{2} G(\tau,\tau')G(\tau',\tau) \right] \mathrm{d}\tau \mathrm{d}\tau'  $$

In the large-$N$ limit, the path integral is dominated by the saddle point, which is found by extremizing the above action. To further simplify, we (legitimately) assume that the saddle point is translation invariant in time and let 
$$ G(\tau,\tau') = G(\tau - \tau') \,,\, \Sigma(\tau,\tau') = \Sigma(\tau - \tau') $$
Note that $G(\tau)$ and $\Sigma(\tau)$ are both anti-periodic. Then the log determinant can be written as a sum over fermionic Matsubara frequencies 
$$ \ln \det(\partial_\tau - \Sigma) = \sum_{\omega} \ln (-\mathrm{i} \omega - \Sigma_{\omega}) $$
where $$ \Sigma_{\omega} = \int_{-\beta/2}^{\beta/2} \Sigma(\tau) e^{\mathrm{i} \omega \tau} \mathrm{d} \tau  $$
Recall also that the fermionic Matsubara frequencies are 
$$ \omega = (2k+1) \pi T \,,\, k \in \mathbb{Z} \,.$$

- Show that the saddle point solution for $G$ and $\Sigma$ satisfies the following (Schwinger-Dyson) equations 
$$ G_\omega = \frac1{-\mathrm{i} \omega - \Sigma_{\omega}} \,,\, \Sigma_\tau = J^2 G_\tau \,.  $$

- Solve for $G_\omega$ analytically, assuming that as a function of $\omega \in \mathbb{C}$, it is analytical everywhere away from the imaginary axis (this follows from the general Lehman representation of correlation functions). 

- When temperature is low, $\beta J \gg 1$, show that

$$  G(\tau) \sim |\tau|^{-1} \mathrm{sign}(\tau) \,,\, 1 \ll J |\tau| \ll J \beta $$

Conclude that the system is gapless with

$$ \Delta_{\psi} = 1/2. $$ 

Could we have guessed that?

- (Optional) We could have calculated $G$ by diagonalizing the matrix $J_{ij}$. It is a GUE random matrix, so its  spectrum obeys the semi-circle law. Where do you see the semicircle law from the above calculation?

- (Optional) Add a chemical potential, $H \to H + \mu \sum_i c_i^\dagger c_i$. For which values of $\mu$ is the system gapless/gapped?

In [ ]:
def fft(x, beta):
    """ from time to frequency """
    return np.fft.ifft(x) * beta
def ifft(x, beta):
    """ from frequency to time """
    return np.fft.fft(x) / beta

In [ ]:
# J = 1 in the code
beta = 200
N = 2 ** 13 # discretization of the thermal circle
ws = (np.arange(N))
ws[N//2:] -= N
ws = ws / (beta * 2) * np.pi * 2
Gw = 1.j * (-  ws + np.sign(ws) * (4 + ws ** 2) ** .5)/2
Gw[::2] = 0 # only fermionic Matsubaras
Gtau = ifft(Gw,beta).real
ts = np.arange(N) / N * beta * 2
plt.plot(ts, Gtau)
plt.xlabel(r"$J \tau$")
plt.ylabel(r"$G(\tau)$")

In [ ]:
plt.loglog(ts[:N//2], Gtau[:N//2])
plt.xlabel(r"$J \tau$")
plt.ylabel(r"$G(\tau)$")
xs = np.linspace(1, 50, 10)
plt.plot(xs, xs ** -1, "--", c = "k")


In [ ]:
ws = np.linspace(-2,2, 200) 
Gw =  (4 - ws ** 2) ** .5 / np.pi / 2
plt.plot(ws, Gw, c ="b", label="$\mathrm{Im}\, G(\mathrm{i} \omega + \epsilon) / \pi$")
N = 128
hsum = 0
stat = 0
for j in range(100):
    Jij = rnd.randn(N, N) +  rnd.randn(N, N) * 1.j
    Jij = (Jij + np.conj(Jij.T)) / 2 / N**.5
    w = la.eigvalsh(Jij)
    h,b = np.histogram(w, bins = np.linspace(-2,2,17), density=1)
    hsum = hsum + h
    stat += 1
plt.plot((b[1:] + b[:-1])/2, hsum / stat, "o", c = 'r', label="density of states of $J_{ij}$")
plt.legend()
plt.xlabel(r"$\omega$")
plt.ylabel(r"$\rho(\omega)$")

## Sachdev-Ye-Kitaev model

The (complex) SYK Hamiltonian is a four-fermion interacting generalization of the random matrix Hamiltonian above: 

\begin{equation} H_4 = \sum_{ijk\ell} J_{ijk\ell} c_i^\dagger c_j^\dagger c_k c_\ell \end{equation}

where the coupling constants are random complex Gaussian with zero mean and such that

$$  \overline{J_{ijk\ell}  J_{\ell k j i}} = \frac{J^2}{4 N^3}  $$

and all other covariances vanish.

- Adapting the above method, derive the effective action, and the SD equations:
$$ G_\omega = \frac1{-\mathrm{i}\omega - \Sigma_\omega} \,,\, \Sigma(\tau) = J^2 G(\tau)^2  (-G(-\tau))$$

These SD equations cannot be solved analytically in general. Nevertheless, we can solve them numerically by iteration. 

In [ ]:
# J = 1
beta = 200
N = 2 ** 15 # discretization of the thermal circle
ws = (np.arange(N))
ws[N//2:] -= N
ws = ws / (beta * 2) * np.pi * 2
Gt = np.zeros(N)
x= .2 # Kitaev's trick to ensure convergence
for j in range(200):
    Sigt = Gt * Gt * (-np.roll(Gt[::-1], 1))
    Sigw = fft(Sigt, beta)
    Gw = 1/(-1.j * ws - Sigw)
    Gw[::2]=0
    Gtnew = ifft(Gw, beta)
    diff = la.norm(Gt - Gtnew) / N**.5 * beta**.5
    if diff < 1e-3:
        print("converged at step %d" % j )
        break
    Gt = Gt * (1-x) + Gtnew * x

In [ ]:
ts = np.arange(N) / N * beta * 2
plt.plot(ts, Gt)
plt.xlabel(r"$J \tau$")
plt.ylabel(r"$G(\tau)$")

In [ ]:
ts = np.arange(N)/N*beta*2
plt.loglog(ts[1:N//2], Gt[1:N//2])
plt.xlabel(r"$J \tau$")
plt.ylabel(r"$G(\tau)$")
xs = np.linspace(1, 50, 10)
plt.plot(xs, xs ** -.5, "--", c = "k", label=r"$\tau^{-1/2}$")
plt.legend()


### Conformal limit

- What do you observe from the solution at large $J\beta$? 

The solution at large $J\beta$ can be understood analytically, at least in a good approximation. The key point is to surmise that in the SD equation 

$$G_\omega^{-1} = -\mathrm{i}\omega - \Sigma_\omega $$

the term $\Sigma_\omega$ dominates the RHS for $\omega$ small. Then we can ignore the kinetic term $ -\mathrm{i}\omega$, and study the approximate SD equations 

$$G_\omega^{-1} = - \Sigma_\omega  \,,\, \Sigma(\tau) = J^2 G(\tau)^2  (-G(-\tau)) =  
J^2 G(\tau)^3$$

where we also assumed $G(-\tau) = -G(\tau)$ which is correct with no chemical potential (this is a particle hole symmetry). See numerics above. 

- Find a power law solution of type

$$ G(\tau) \propto \mathrm{sign}(\tau) |\tau|^{-2\Delta} $$

to the approximate SD equation (you can assume $\beta\to\infty$). Determine 

$$\Delta = \frac14 \,. $$

Could we have guessed that from the Hamiltonian?

- Verify that the assumption behind the approximation is correct (thus we found a self-consistent approximate solution that explains the numerics).

- Does the same method work for the random matrix free-fermion model above? Can you think other similar models where a similar procedure works.


### Fragility of SYK

Consider a linear combination of the random matrix Hamiltonian and the SYK Hamiltonian 

$$ H = H_2[J\to J_2] + H_4[J\to J_4] \,, $$

where the random couplings from the two terms are independent. 

- What are the SD equations?
- At low temperature, show that $H_2$ dominates, and that the effect of $H_4$ can be studied perturbatively. 